In [1]:
import pandas as pd
import urllib
from bs4 import BeautifulSoup
import time
import re

In [2]:
APIKeyZ = "api_key=4074d0170761c40d9c07d9016ddd4965"
base_url_search = "https://api.themoviedb.org/3/discover/movie?"
popular_desc = "&sort_by=popularity.desc"
year =  "&primary_release_year={}"
# access different pages
# https://api.themoviedb.org/3/discover/movie?api_key=4074d0170761c40d9c07d9016ddd4965&page=2&sort_by=popularity.desc
page_number = '&page={}'
query_url = 'https://api.themoviedb.org/3/movie/{}?'

In [3]:
# get top 500 popular movies ID, to be used for further search of IMDB ID
number_page = 0
movie_id_list = []
for i in range(1, 26):
    popular_movies =  base_url_search + APIKeyZ + popular_desc + page_number.format(i)
    page = urllib.urlopen(popular_movies).read()
    soup = BeautifulSoup(page, "lxml")
    prettified = soup.prettify()
    movie_list = [m.start() for m in re.finditer('"id"', prettified)] # this finds all indexes of occurences of "id"
    
    for j in range(len(movie_list)):
        j_beginning = 5
        movie_id_temp = ''
        while (prettified[movie_list[j] + j_beginning].isdigit()):
            movie_id_temp += str(prettified[movie_list[j] + j_beginning])
            j_beginning += 1
        movie_id_list += [int(movie_id_temp)]
    if i % 40 == 0:
        time.sleep(10)
    

In [4]:
# query each movie by TMDB ID and get IMDB ID
movie_id_list_IMDB = []
for i in range(len(movie_id_list)):
    page_url = query_url.format(movie_id_list[i]) + APIKeyZ
    page = urllib.urlopen(page_url).read()
    soup = BeautifulSoup(page, "lxml")
    prettified = soup.prettify()
    imdb_id_index = prettified.find('"imdb_id"')
    j_beginning = 13
    movie_id_temp = ''
    while (prettified[prettified.find('"imdb_id"') + j_beginning].isdigit()):
        movie_id_temp += str(prettified[prettified.find('"imdb_id"') + j_beginning])
        j_beginning += 1
    movie_id_list_IMDB += [movie_id_temp]
    if i % 40 == 39:
        time.sleep(10.1)

In [5]:
# Write out movie TMDB IDs
thefile = open('popular_tmdb_id2.txt', 'w')
for item in movie_id_list:
    thefile.write("%s\n" % item)
thefile.close()

In [6]:
# Write out movie IMDB IDs 
thefile = open('popular_imdb_id2.txt', 'w')
for item in movie_id_list_IMDB:
    thefile.write("%s\n" % item)
thefile.close()

In [7]:
# Use package tmdbsimple to extract data
import tmdbsimple as tmdb
# use "!pip install tmdbsimple" to install
tmdb.API_KEY = '4074d0170761c40d9c07d9016ddd4965'

In [8]:
# search one movie by ID and create list of column names
movie = tmdb.Movies(movie_id_list[0])
response = movie.info()
columns = []
for i in range(len(response)):
    column_temp = [str(response.items()[i][0])]
    columns += column_temp

In [9]:
# create a dataframe to store information of top 500 popular movies
index = range(0, len(movie_id_list)) 
df = pd.DataFrame(index = index, columns=columns)
df = df.fillna(0)

In [13]:
for i in range(len(index)):
    movie = tmdb.Movies(movie_id_list[i])
    response = movie.info()
    if i % 40 == 39:
        time.sleep(11)
    for j in range(0, len(columns)):
        if j == 1:
            country_temp = []
            for k in range(len(response[columns[j]])):
                country_temp += [(response[columns[j]][k]['name'])]
            df.iloc[i,j] = str(country_temp)
        elif j == 6:
            genre_temp = []
            for k in range(len(response[columns[j]])):
                genre_temp += [(response[columns[j]][k]['name'])]
            df.iloc[i,j] = str(genre_temp)
        elif j == 11:
            if response[columns[j]] != None:
                df.iloc[i,j] = str(response[columns[j]])
        elif j == 14:
            language_temp = []
            for k in range(len(response[columns[j]])):
                language_temp += [(response[columns[j]][k]['iso_639_1'])]
            df.iloc[i,j] = str(language_temp)
        elif j == 18:
            company_temp = []
            for k in range(len(response[columns[j]])):
                company_temp += [(response[columns[j]][k]['name'])]
            df.iloc[i,j] = str(company_temp)
        else:
            df.iloc[i,j] = response[columns[j]]
        

In [15]:
df

,poster_path,production_countries,revenue,overview,video,id,genres,title,tagline,vote_count,...,imdb_id,adult,backdrop_path,production_companies,release_date,popularity,original_title,budget,vote_average,runtime
0,/tWqifoYuwLETmmasnGHO7xBjEtt.jpg,[u'United States of America'],877017934,A live-action adaptation of Disney's version o...,False,321612,"[u'Fantasy', u'Music', u'Romance']",Beauty and the Beast,Be our guest.,1274,...,tt2771200,False,/6aUWe0GSl69wMTSWWexsorMIvwU.jpg,"[u'Walt Disney Pictures', u'Mandeville Films']",2017-03-17,179.451320,Beauty and the Beast,160000000,7.1,129
1,/45Y1G5FEgttPAwjTYic6czC9xCn.jpg,[u'United States of America'],586061420,"In the near future, a weary Logan cares for an...",False,263115,"[u'Action', u'Drama', u'Science Fiction']",Logan,His Time Has Come,2104,...,tt3315342,False,/5pAGnkFYSsFJ99ZxDIYnhQbQFXs.jpg,"[u'Twentieth Century Fox Film Corporation', u""...",2017-02-28,116.369877,Logan,97000000,7.6,141
2,/s9ye87pvq2IaDvjv9x4IOXVjvA7.jpg,[u'United States of America'],601303829,A koala named Buster recruits his best friend ...,False,335797,"[u'Animation', u'Comedy', u'Drama', u'Family',...",Sing,Auditions begin 2016.,1015,...,tt3470600,False,/fxDXp8un4qNY9b1dLd7SH6CKzC.jpg,"[u'Universal Pictures', u'Fuji Television Netw...",2016-11-23,79.989984,Sing,75000000,6.7,108
3,/aoUyphk4nwffrwlZRaOa0eijgpr.jpg,[u'United States of America'],478935658,Explore the mysterious and dangerous home of t...,False,293167,"[u'Science Fiction', u'Action', u'Adventure', ...",Kong: Skull Island,All hail the king,883,...,tt3731562,False,/pGwChWiAY1bdoxL79sXmaFBlYJH.jpg,"[u'Warner Bros.', u'Legendary Entertainment']",2017-03-08,60.933195,Kong: Skull Island,190000000,6.1,118
4,/jjBgi2r5cRt36xF6iNUEhzscEcb.jpg,[u'United States of America'],1513528810,Twenty-two years after the events of Jurassic ...,False,135397,"[u'Action', u'Adventure', u'Science Fiction', ...",Jurassic World,The park is open.,6720,...,tt0369610,False,/dkMD5qlogeRMiEixC4YNPUvax2T.jpg,"[u'Universal Studios', u'Amblin Entertainment'...",2015-06-09,50.976724,Jurassic World,150000000,6.5,124
5,/myRzRzCxdfUWjkJWgpHHZ1oGkJd.jpg,[u'United States of America'],60100336,"In the near future, Major is the first of her ...",False,315837,"[u'Action', u'Drama', u'Science Fiction']",Ghost in the Shell,,244,...,tt1219827,False,/lsRhmB7m36pEX0UHpkpJSE48BW5.jpg,"[u'Paramount Pictures', u'DreamWorks SKG', u'G...",2017-03-29,41.800777,Ghost in the Shell,110000000,6.4,106
6,/gri0DDxsERr6B2sOR1fGLxLpSLx.jpg,"[u'United Kingdom', u'United States of America']",809342332,"In 1926, Newt Scamander arrives at the Magical...",False,259316,"[u'Adventure', u'Action', u'Fantasy']",Fantastic Beasts and Where to Find Them,From J.K. Rowling's wizarding world.,3122,...,tt3183660,False,/kMzU4PkXcKcDMngCxXji0BbVXsu.jpg,"[u'Heyday films', u'Warner Bros.']",2016-11-16,39.553052,Fantastic Beasts and Where to Find Them,180000000,7.0,133
7,/67NXPYvK92oQgEQvLppF2Siol9q.jpg,[u'United States of America'],111065699,A story about how a new baby's arrival impacts...,False,295693,"[u'Animation', u'Comedy', u'Family']",The Boss Baby,Born leader,182,...,tt3874544,False,/8keMlLuzB9XIUBnbdEq5DCqZdHQ.jpg,"[u'Twentieth Century Fox Film Corporation', u'...",2017-03-23,38.723910,The Boss Baby,0,5.8,97
8,/nBNZadXqJSdt05SHLqgT0HuC5Gm.jpg,"[u'Canada', u'United States of America', u'Uni...",675120017,Interstellar chronicles the adventures of a gr...,False,157336,"[u'Adventure', u'Drama', u'Science Fiction']",Interstellar,Mankind was born on Earth. It was never meant ...,8066,...,tt0816692,False,/xu9zaAevzQ5nnrsXN6JcahLnG4i.jpg,"[u'Paramount Pictures', u'Legendary Pictures',...",2014-11-05,34.594463,Interstellar,165000000,8.0,169
9,/z09QAf8WbZncbitewNk6lKYMZsh.jpg,[u'United States of America'],1028131226,Dory is reunited with her friends Nemo and Mar...,False,127380,"[u'Adventure', u'Animation', u'Comedy', u'Fami...",Finding Dory,An unforgettable journey she probably won't re...,2914,...,tt2277860,False,/iWRKYHTFl

In [16]:
df.to_csv('TMDb_data.txt', encoding='utf-8')

---------------